In [154]:
import time #import the time module. Used for adding pauses during operation
import numpy as np
from Arm_Lib import Arm_Device #import the module associated with the arm
import FK, IK
import smbus

Arm = Arm_Device() # Get DOFBOT object
bus = smbus.SMBus( 1 )
ANGLE_READ_REG_ID = 0x37
I2C_ID = 0x15
JOINT_SCALING_FACTOR = (
    [
        700.0,
        -700.0,
        -700.0,
        -700.0,
        700.0,
        700.0
    ]
)

JOINT_OFFSET = (
    [
        2000.0,
        2000.0,
        2000.0,
        2000.0,
        1466.0,
        2000.0,
    ]
)
time.sleep(.2) #this pauses execution for the given number of seconds
for joint in range(5):
    print(read_angle(bus, joint + 1))

90.24555334077036
43.42671636722308
34.58679609949038
66.26317705886589
16.661402223254626


In [39]:
def read_angle(bus, num):
    bus.write_byte_data( I2C_ID, ANGLE_READ_REG_ID, num )
    time.sleep(0.1)
    raw = bus.read_word_data(I2C_ID, ANGLE_READ_REG_ID)
    raw1 = ( raw >> 8 & 0xff ) | ( raw << 8 & 0xff00 )
    return ( raw1 - JOINT_OFFSET[ num - 1 ] ) / JOINT_SCALING_FACTOR[ num - 1 ] * 180 / np.pi + 90

In [33]:
def moveJoint(jnum,ang,speedtime):
    """
    function used to move the specified joint to the given position
    moveJoint(jnum, ang, speedtime) moves joint jnum to position ang degrees in speedtime milliseconds
    function returns nothing
    """
    # call the function to move joint number jnum to ang degrees in speedtime milliseconds
    Arm.Arm_serial_servo_write(jnum,ang,speedtime)
    return

In [34]:
def moveJoints(ang,speedtime):
    """
    function used to move the specified joint to the given position
    moveJoint(jnum, ang, speedtime) moves joint jnum to position ang degrees in speedtime milliseconds
    function returns nothing
    """
    # call the function to move joint number jnum to ang degrees in speedtime milliseconds
    for i in range(5):
        Arm.Arm_serial_servo_write(i + 1,ang[i],speedtime)
        time.sleep(0.1)  
    return

In [150]:
def followJointPath(q, speedtime, pid):
    # q in shape of (5, num_steps), q is in radians
    # pid is a tuple of form(kp, ki, kd) if given
    e = np.zeros((5))
    e_i = np.zeros((5))
    e_d = np.zeros((5))
    e_p = np.zeros((5))
    u_pid = np.zeros((5))
    N = np.shape(q)[1]
    q_measures = np.zeros((5, N))
    q_errors = np.zeros((5, N))
    for step in range(N):
        u_pid = pid[0] * e + pid[1] * e_i + pid[2] * e_d
        moveJoints(q[:, step] + u_pid, speedtime)
        time.sleep(1) #add a pause to allow time for joints to move
        for joint in range(5):
            q_measures[joint, step] = read_angle(bus, joint + 1) #read the actual position of the desired joint
        e = q[:, step] - q_measures[:, step]
        e_i += e / N
        e_d = (e - e_p) * N
        e_p = e
        q_errors[:, step] = e
        print(q_measures[:, step], e, sum(abs(e)))
        time.sleep(0.1)
        #input()
    return q_measures, q_errors

In [151]:
# Forward kinematics
n = 5
joint_angles = np.array([[90, 90, 80, 90, 90],
                         [90, 45, 135, 45, 135],
                         [28, 35, 150, 160, 78],
                         [130, 120, 50, 120, 125],
                         [90, 45, 35, 66, 15]])
for i in range(n):
    q = joint_angles[i, :] * np.pi / 180
    ROT, POT = FK.fwkin_POE_Dofbot(q)
    q_measures, qerror = followJointPath(np.reshape(joint_angles[i], (-1, 1)), 500, pid = (0, 0, 0))
    input()

[ 89.50889332  89.59074443  80.3415686   90.          92.20998007] [ 0.49110668  0.40925557 -0.3415686   0.         -2.20998007] 3.45191091279


[  89.50889332   43.99967416  134.19960134   45.14558975  136.49143252] [ 0.49110668  1.00032584  0.80039866 -0.14558975 -1.49143252] 3.92885345233


[  28.52981369   34.83234944  150.07871738  159.81899989   80.25971748] [-0.52981369  0.16765056 -0.07871738  0.18100011 -2.25971748] 3.21689921911


[ 129.28853452  120.36676314   50.38406102  120.77601871  126.26004332] [ 0.71146548 -0.36676314 -0.38406102 -0.77601871 -1.26004332] 3.49835167137


[ 90.32740445  43.42671637  34.66864721  66.26317706  16.74325334] [-0.32740445  1.57328363  0.33135279 -0.26317706 -1.74325334] 4.23847126977


In [152]:
q_measures, qerror = followJointPath(np.transpose(joint_angles), 500, pid = (0, 0, 0))

[ 90.32740445  89.50889332  79.11380189  89.26333998  90.98221336] [-0.32740445  0.49110668  0.88619811  0.73666002 -0.98221336] 3.42358262878
[  90.40925557   43.99967416  133.87219688   45.14558975  136.49143252] [-0.40925557  1.00032584  1.12780312 -0.14558975 -1.49143252] 4.1744067931
[  28.52981369   34.75049833  149.91501515  159.81899989   80.17786637] [-0.52981369  0.24950167  0.08498485  0.18100011 -2.17786637] 3.223166696
[ 129.45223675  120.36676314   50.38406102  120.77601871  126.34189443] [ 0.54776325 -0.36676314 -0.38406102 -0.77601871 -1.34189443] 3.41650055778
[ 90.32740445  43.50856748  34.5867961   66.26317706  16.66140222] [-0.32740445  1.49143252  0.4132039  -0.26317706 -1.66140222] 4.15662015618


In [153]:
np.save("q_measurement_FK.npy", q_measures)
np.save("q_error_FK.npy", q_errors)

In [146]:
moveJoints([90, 90, 90, 90, 270], 100)

In [129]:
moveJoint(5, 90, 100)

In [19]:
# Inverse kinematics
n = 5
joint_angles = np.array([[90, 45, 135, 45, 135],
                        [180, 90, 90, 90, 90],
                        [90, 45, 135, 45, 135],
                        [90, 45, 135, 45, 135],
                        [90, 45, 135, 45, 135]])
for i in range(n):
    q = joint_angles[i, :] * np.pi / 180
    ROT, POT = FK.fwkin_POE_Dofbot(q)
    q_prime = IK.invkin_subproblems_Dofbot(ROT, POT)
    print("Caculated:", joint_angles[i, :], "\n", "Computed by IK:", q_prime)

Caculated: [ 90  45 135  45 135] 
 Computed by IK: [[ -90.   90.  -90.   90.]
 [  90.   45.  135.   90.]
 [ 135.  135.   45.   45.]
 [  90.   45.  135.   90.]
 [ -45.  135.  -45.  135.]]
Caculated: [180  90  90  90  90] 
 Computed by IK: [[  0.   0.]
 [ 90.  90.]
 [ 90.  90.]
 [ 90.  90.]
 [  0.   0.]]
Caculated: [ 90  45 135  45 135] 
 Computed by IK: [[ -90.   90.  -90.   90.]
 [  90.   45.  135.   90.]
 [ 135.  135.   45.   45.]
 [  90.   45.  135.   90.]
 [ -45.  135.  -45.  135.]]
Caculated: [ 90  45 135  45 135] 
 Computed by IK: [[ -90.   90.  -90.   90.]
 [  90.   45.  135.   90.]
 [ 135.  135.   45.   45.]
 [  90.   45.  135.   90.]
 [ -45.  135.  -45.  135.]]
Caculated: [ 90  45 135  45 135] 
 Computed by IK: [[ -90.   90.  -90.   90.]
 [  90.   45.  135.   90.]
 [ 135.  135.   45.   45.]
 [  90.   45.  135.   90.]
 [ -45.  135.  -45.  135.]]


In [110]:
# Joint space path planing and following using Jacobian IK (without/with pid control on arm movement)
q_lambda = np.load("horizontal_jointpath.npy")
#q_lambda = np.load("horizontal_jointpath.npy")
q_lambda = q_lambda * 180 / np.pi
speedtime = 10
#q_measures_pid = followJointPath(q_lambda, speedtime, pid)
pid = (0.8, 0.1, 0)
pid = (0, 0, 0)
q_measures, q_errors = followJointPath(q_lambda, speedtime, pid)

[ 134.52700579   89.01778664   19.77174454   19.28063786  272.03687662] [ 0.47299421  0.98221336  0.22825546  0.71936214 -2.03687662] 4.43970179666
[ 129.7796412    57.58695902   77.39492851  -24.91896348  272.11872774] [-0.26786753  0.95510318  1.43614075 -0.27563205 -2.11872774] 5.05347124744
[ 123.55895657   59.22398129   77.31307739  -29.2570725   272.20057885] [-0.47978309  1.552502    0.34341649 -0.33139452 -2.20057885] 4.90767495912
[ 116.19235635   60.94285468   77.06752405  -32.61296816  272.20057885] [-0.54935307  1.56880006 -0.3653247  -0.47230897 -2.20057885] 5.15636565712
[ 107.67984054   61.6795147    76.08531069  -35.15035268  272.20057885] [-0.41988317  2.00589463 -0.10670677 -0.451907   -2.20057885] 5.18497042898
[  98.4306647    62.49802583   75.92160846  -36.70552384  272.20057885] [-0.27562948  1.81646107 -0.35850988 -0.29885185 -2.20057885] 4.95003112057
[  89.26333998   62.66172806   75.83975735  -36.70552384  272.20057885] [-0.53915448  1.73911016 -0.3348155  -0.

In [111]:
np.sum(abs(q_errors), axis=0)

array([ 4.4397018 ,  5.05347125,  4.90767496,  5.15636566,  5.18497043,
        4.95003112,  5.30835485,  4.77093824,  4.34665123,  5.44280154,
        5.62823384])

In [112]:
np.save("q_measurement_without_pid_ypath.npy", q_measures)
np.save("q_error_without_pid_ypath.npy", q_errors)

In [135]:
# Object avoidance path in task space -> joint space path
q_lambda = np.load("objavoidp.npy")
q_lambda = q_lambda * 180 / np.pi
speedtime = 10
pid = (0.8, 0.1, 0)
pid = (0, 0, 0)
q_measures, q_errors = followJointPath(q_lambda, speedtime, pid)

[ 134.36330357   89.18148886   19.77174454   -0.60918274   90.98221336] [ 0.63669643  0.81851114  0.22825546  0.60918274 -0.98221336] 3.27485913772
[ 134.36330357   77.72233296   34.75049833  -11.65908308   90.98221336] [-0.4453955   0.851739    1.4135454  -0.37404545 -0.98221336] 4.06693870179
[ 133.3810902    77.72233296   34.75049833  -12.15018976   90.98221336] [-0.5581164   1.21018331  1.19233272 -0.6064933  -0.98221336] 4.54933909345
[ 132.31702573   77.72233296   34.75049833  -13.13240312   90.98221336] [-0.60101783  1.54843331  0.97963504 -0.31459018 -0.98221336] 4.42588971701
[ 131.17111014   77.72233296   34.75049833  -13.54165869   90.98221336] [-0.57323027  1.86624186  0.77464119 -0.56443367 -0.98221336] 4.76076035425
[ 130.10704566   77.80418408   34.75049833  -14.36016983   90.98221336] [-0.63752831  2.05386888  0.53203227 -0.41121088 -0.98221336] 4.61685370652
[ 128.79742784   77.88603519   34.75049833  -14.7694254    90.98221336] [-0.4655077   2.25736944  0.35050813 -0.

In [136]:
np.save("q_measurement_without_pid_yavoidpath.npy", q_measures)
np.save("q_error_without_pid_yavoidpath.npy", q_errors)